In [1]:
import importlib
import types
from eth2spec.config.config_util import prepare_config
from eth2spec.utils.ssz.ssz_impl import hash_tree_root

import os, sys
# Load current beacon runner specs (v1.1.0-alpha.3)
sys.path.insert(1, os.path.realpath(os.path.pardir))

import beaconrunner as br

prepare_config(".", "fast.yaml")

br.reload_package(br)

In [2]:
def extract_state(s):
    validators = s["network"].validators
    validator = validators[1]
    head = br.specs.get_head(validator.store)
    current_state = validator.store.block_states[head].copy()
    return current_state

def current_slot(params, step, sL, s, _input):
    return ("current_slot", s["network"].validators[0].data.slot)

def total_balance_asap(params, step, sL, s, _input):
    validators = s["network"].validators
    current_state = extract_state(s)
    current_epoch = br.specs.get_current_epoch(current_state)
    asap_indices = [i for i, v in enumerate(validators) if v.validator_behaviour == "asap"]
    asap_balances = [b for i, b in enumerate(current_state.balances) if i in asap_indices]
    return ("total_balance_asap", sum(asap_balances))

def total_balance_malicious(params, step, sL, s, _input):
    validators = s["network"].validators
    current_state = extract_state(s)
    current_epoch = br.specs.get_current_epoch(current_state)
    malicious_indices = [i for i, v in enumerate(validators) if v.validator_behaviour == "malicious"]
    malicious_balances = [b for i, b in enumerate(current_state.balances) if i in malicious_indices]
    return ("total_balance_malicious", sum(malicious_balances))

def get_base_reward(params, step, sL, s, _input):
    current_state = extract_state(s)
    base_reward = br.specs.get_base_reward(current_state, 0)
    return ("base_reward", base_reward)

def get_block_proposer(params, step, sL, s, _input):
    current_state = extract_state(s)
    block_proposer = [v.validator_index for v in s["network"].validators if v.data.current_proposer_duties[s["current_slot"] % br.specs.SLOTS_PER_EPOCH]][0]
    return ("block_proposer", block_proposer)

def get_block_proposer_balance(params, step, sL, s, _input):
    current_state = extract_state(s)
    block_proposer_balance = current_state.balances[s["block_proposer"]]
    return ("block_proposer_balance", block_proposer_balance)

def get_sync_committee(params, step, sL, s, _input):
    current_state = extract_state(s)
    current_epoch = br.specs.get_current_epoch(current_state)
    sync_committee = current_state.current_sync_committee
    sync_committee = br.specs.get_sync_committee_indices(current_state, current_epoch)
    return ("sync_committee", sync_committee)

def get_head(params, step, sL, s, _input):
    validators = s["network"].validators
    validator = validators[0]
    head = br.specs.get_head(validator.store).hex()[0:6]
    return ("head", head)

def get_malicious_head(params, step, sL, s, _input):
    malicious_head = s['malicious_data'].malicious_head
    if malicious_head is None:
        return ('malicious_head', "None")
    
    return ('malicious_head', malicious_head.hex()[0:6])

def get_current_validator_state(params, step, sL, s, _input):
    current_state = extract_state(s)
    current_validator_state = []
    for v in s["network"].validators:
        current_validator_state += [{
            "slot": v.data.slot,
            "validator_index": v.validator_index,
            "balance": current_state.balances[v.validator_index],
            "block_proposer": 1 if s["block_proposer"] == v.validator_index else 0,
            "attester": 1 if v.data.current_attest_slot == v.data.slot else 0,
            "sync_committee": len(v.data.current_sync_committee),
        }]
    return ("current_validator_state", current_validator_state)

observers = {
    "current_slot": current_slot,
    "total_balance_asap": total_balance_asap,
    "total_balance_malicious": total_balance_malicious,
    "base_reward": get_base_reward,
    "block_proposer": get_block_proposer,
    "block_proposer_balance": get_block_proposer_balance,
#     "sync_committee": get_sync_committee,
    "head": get_head,
    "malicious_head": get_malicious_head,
    "current_validator_state": get_current_validator_state,
}

In [3]:
from random import sample
from beaconrunner.validators.ASAPValidator import ASAPValidator
from beaconrunner.validators.MaliciousValidator import MaliciousValidator
import numpy as np

rng = np.random.default_rng(42)

def simulate_once(network_sets, num_run, num_validators, network_update_rate):
    
    num_malicious = int((3 * num_validators) / 10)

    # We sample the position on the p2p network of prudent validators randomly
    malicious_set = set(
        rng.choice(np.arange(num_validators), size=num_malicious, replace=True)
    )
    print("malicious_indices", malicious_set)

    validators = []
    malicious_validator_indices = []

    # Initiate validators
    for i in range(num_validators):
        if i in malicious_set:
            new_validator = MaliciousValidator(i)
            malicious_validator_indices.append(i)
        else:
            new_validator = ASAPValidator(i)
        validators.append(new_validator)
    
    # Create a genesis state
    (genesis_state, genesis_block) = br.simulator.get_genesis_state_block(validators)

    # Validators load the state
    [v.load_state(genesis_state.copy(), genesis_block.copy()) for v in validators]
    
    # We skip the genesis block
    br.simulator.skip_genesis_block(validators)

    network = br.network.Network(validators=validators, sets=network_sets)
    malicious_data = br.malicious.MaliciousData(
        malicious_validator_indices=malicious_validator_indices
    )
    
    parameters = br.simulator.SimulationParameters({
        "num_epochs": 1,
        "num_run": num_run,
        "run_index": 1,
        "frequency": 1,
        "network_update_rate": network_update_rate,
    })

    return br.simulator.malicious_simulate(network, malicious_data, parameters, observers)

In [4]:
num_validators = 12

# Create the network peers
set_a = br.network.NetworkSet(validators=list(range(num_validators)))
network_sets = list([set_a])

num_runs = 1
network_update_rate = 0.25

#df = pd.concat([simulate_once(network_sets, num_run, num_validators, network_update_rate) for num_run in range(num_runs)])
df = simulate_once(network_sets, num_runs, num_validators, network_update_rate)

malicious_indices {1, 9, 7}
will simulate 1 epochs ( 4 slots ) at frequency 1 moves/second
total 48 simulation steps

                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (48, 2, 1, 11)


Initializing configurations:   0%|          | 0/1 [00:00<?, ?it/s]

Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
9 (malicious) proposing block for slot 1
10 proposing block for slot 2
4 proposing block for slot 3
5 proposing block for slot 4


Flattening results:   0%|          | 0/241 [00:00<?, ?it/s]

Total execution time: 4.87s


In [5]:
df = df.drop(columns=['network', 'malicious_data'])

In [6]:
import pandas as pd
pd.set_option('display.max_rows', 10)
df.head(10)

,current_slot,total_balance_asap,total_balance_malicious,base_reward,block_proposer,block_proposer_balance,head,malicious_head,current_validator_state,simulation,subset,run,substep,timestep
0,1,288000000000,96000000000,3304928,9,32000000000,2281a3,None,"[{'slot': 1, 'validator_index': 0, 'balance': ...",0,0,1,0,0
1,1,288000000000,96000000000,3304928,9,32000000000,2281a3,None,"[{'slot': 1, 'validator_index': 0, 'balance': ...",0,0,1,1,1
2,1,288000000000,96000000000,3304928,9,32000000000,2281a3,None,"[{'slot': 1, 'validator_index': 0, 'balance': ...",0,0,1,2,1
3,1,288000000000,96000000000,3304928,9,32000000000,2281a3,3b1e64,"[{'slot': 1, 'validator_index': 0, 'balance': ...",0,0,1,3,1
4,1,288000000000,96000000000,3304928,9,32000000000,2281a3,3b1e64,"[{'slot': 1, 'validator_index': 0, 'balance': ...",0,0,1,4,1
5,1,288000000000,96000000000,3304928,9,32000000000,2281a3,3b1e64,"[{'slot': 1, 'validator_index': 0, 'balance': ...",0,0,1,5,1
6,1,288000000000,96000000000,3304928,9,32000000000,2281a3,None,"[{'slot': 1, 'validator_index': 0, 'balance': ...",0,0,1,1,2
7,1,288000000000,96000000000,3304928,9,32000000000,2281a3,None,"[{'slot': 1, 'validator_index': 0, 'balance': ...",0,0,1,2,2
8,1,288000000000,96000000000,3304928,9,32000000000,2281a3,None,"[{'slot': 1, 'validator_index': 0, 'balance': ...",0,0,1,3,2
9,1,288000000000,96000000000,3304928,9,32000000000,2281a3,None,"[{'slot': 1, 'validator_index': 0, 'balance': ...",0,0,1,4,2
